In [1]:
import geopandas as gpd
import pandas as pd
import sqlite3
import rasterio
from rasterio.transform import from_origin
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import fnmatch
import os

In [7]:
fire_suffix="_fire"
directory = 'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/outs/'
year = 7  
pattern = "out_*_fire.db" 
out_raster = f'C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/outs/rasters/biomass_year{year}{fire_suffix}.tif'

def fetch_data(db_path, year):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    query = f"""
    SELECT latitude, longitude, total_biomass
    FROM Biomass_Output
    WHERE year = {year}
    """
    # Read the query directly into a Pandas DataFrame
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def process_files(directory, year, pattern):
    gdfs = []
    # List all files that match the pattern in the directory
    for filename in os.listdir(directory):
        if fnmatch.fnmatch(filename, pattern):  # Check if filename matches the pattern
            db_path = os.path.join(directory, filename)
            print(db_path)
            df = fetch_data(db_path, year)
            if not df.empty:
                gdf = gpd.GeoDataFrame(
                    df,
                    geometry=gpd.points_from_xy(df.longitude, df.latitude),
                    crs="EPSG:4326"
                )
                gdf = gdf.to_crs("EPSG:3310")  # Reproject to California Albers
                gdfs.append(gdf)
    return gdfs


gdfs = process_files(directory, year,pattern)


def get_union_extent(gdfs):
    """Calculate the union of bounds of multiple geodataframes."""
    xmin = ymin = float('inf')
    xmax = ymax = float('-inf')
    
    for gdf in gdfs:
        minx, miny, maxx, maxy = gdf.total_bounds
        xmin = min(xmin, minx)
        ymin = min(ymin, miny)
        xmax = max(xmax, maxx)
        ymax = max(ymax, maxy)
    
    return xmin, ymin, xmax, ymax


def initialize_raster(xmin, ymax, xmax, ymin, cell_size):
    """Initialize a raster array and return it with its transform."""
    width = int((xmax - xmin) / cell_size) + 1
    height = int((ymax - ymin) / cell_size) + 1
    transform = from_origin(xmin, ymax, cell_size, cell_size)
    raster = np.full((height, width), np.nan)  # Initialize with NaNs
    return raster, transform, width, height

def rasterize_gdf(gdf, raster, transform, width, height):
    """Rasterize geodataframe points into the given raster based on the transform."""
    for point, value in zip(gdf.geometry, gdf['total_biomass']):
        col, row = ~transform * (point.x, point.y)
        col, row = int(col), int(row)
        if 0 <= row < height and 0 <= col < width:
            raster[row, col] = value  # Here we're overwriting; modify for other aggregation logic as needed

# Determine the union extent of all GeoDataFrames
xmin, ymin, xmax, ymax = get_union_extent(gdfs)

# Initialize the master raster
cell_size = 30  # Assuming 30m resolution
raster, transform, width, height = initialize_raster(xmin, ymax, xmax, ymin, cell_size)

# Rasterize each GeoDataFrame
for gdf in gdfs:
    rasterize_gdf(gdf, raster, transform, width, height)

# Save the combined raster to a file
meta = {
    'driver': 'GTiff',
    'dtype': str(raster.dtype),
    'nodata': np.nan,
    'width': width,
    'height': height,
    'count': 1,
    'crs': 'EPSG:3310',
    'transform': transform
}
with rasterio.open(out_raster, 'w', **meta) as dst:
    dst.write(raster, 1)





C:/Users/ingli/OneDrive - University of Nevada, Reno/Desktop/unr/git/RVS/240226_test/outs/out_1_fire.db


In [49]:

# Path to the raster file


In [6]:
# Mask no-data values, assuming no-data is represented as np.nan
raster_data = np.where(raster_data == -9999, np.nan, raster_data)

# Now plot using the masked array
fig, ax = plt.subplots(figsize=(10, 10))
cax = ax.imshow(raster_data, cmap=cmap, interpolation='nearest')
fig.colorbar(cax, ax=ax, label='Total Biomass')
ax.set_title('Total Biomass Raster Plot')
plt.show()


NameError: name 'raster_data' is not defined